In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-06-08 08:42:09.689141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 08:42:09.689417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 08:42:09.844494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/user-in-df/user.csv
/kaggle/input/menu-df/menu.csv


In [2]:
menu_df = pd.read_csv("/kaggle/input/menu-df/menu.csv")

menu_df[menu_df.diet_labels.isna()]
menu_df.fillna(value="Low-Crab", inplace=True)
menu_df

,food_ids,recipe_uri,recipe_name,calories,sugar_content(g),diet_labels,ingredients,image_url,instruction_url
0,"food_auq934abrnrrnramqxvg3b0r2m1i, food_be6ctz...",http://www.edamam.com/ontologies/edamam.owl#re...,Pineapple Angel Food Cake,514.492561,95.821388,"Low-Fat, Low-Sodium","1 20 ounce can of crushed pineapple, 1 Box of ...",https://edamam-product-images.s3.amazonaws.com...,https://food52.com/recipes/84320-pineapple-ang...
1,"food_bgkl0cuasoeomtbolalmcauhha54, food_axi2ij...",http://www.edamam.com/ontologies/edamam.owl#re...,Ginger Berry Compote with Angel Food Cake,1332.868157,76.482700,Low-Fat,"1 bag (1 pound) frozen mixed berries, unthawed...",https://edamam-product-images.s3.amazonaws.com...,https://www.marthastewart.com/338800/ginger-be...
2,"food_al3e7s2a1lm4i6bga7o0abp399zg, food_axi2ij...",http://www.edamam.com/ontologies/edamam.owl#re...,Raspberry Fool with Toasted Angel Food Cake,1677.131024,98.829187,Low-Crab,"2½ cups raspberries (about 12 oz.), 2 Tbsp. pl...",https://edamam-product-images.s3.amazonaws.com...,https://www.bonappetit.com/recipe/raspberry-fo...
3,"food_be6ctzna8a8u4wa9hlv3na5qcvpk, food_al3e7s...",http://www.edamam.com/ontologies/edamam.owl#re...,Raspberry Fool with Toasted Angel Food Cake,1677.131024,98.829187,Low-Crab,"4 cups 1"" pieces angel food cake (from about 1...",https://edamam-product-images.s3.amazonaws.com...,https://www.epicurious.com/recipes/food/views/...
4,"food_ani7vifaibaoiyb7mwh2obf054t3, food_axi2ij...",http://www.edamam.com/ontologies/edamam.owl#re...,Angel Food Cake,1450.164667,203.111750,Low-Fat,"1 cup (130 g) cake flour (not self-rising), 1 ...",https://edamam-product-images.s3.amazonaws.com...,http://www.davidlebovitz.com/2012/08/angel-foo...
...,...,...,...,...,...,...,...,...,...
3549,"food_bz8rcwobbzm7zhb3wh2n7aznivou, food_b2id9o...",http://www.edamam.com/ontologies/edamam.owl#re...,Cook the Book: Grilled Shishito Peppers with S...,331.166474,10.886217,Low-Carb,"1 pound shishito peppers, 2 tablespoons Asian ...",https://edamam-product-images.s3.amazonaws.com...,http://www.seriouseats.com/recipes/2010/08/pho...
3550,"food_bdrxu94aj3x2djbpur8dhagfhkcn, food_bu8t61...",http://www.edamam.com/ontologies/edamam.owl#re...,Asian Chicken Salad with Wasabi Dressing,2508.988858,37.122404,"High-Fiber, Low-Carb","4 skinless, boneless chicken breast halves, ½ ...",https://edamam-product-images.s3.amazonaws.com...,http://www.delish.com/cooking/recipe-ideas/rec...
3551,"food_a1vgrj1bs8rd1majvmd9ubz8ttkg, food_b6ywzl...",http://www.edamam.com/ontologies/edamam.owl#re...,Asian Noodle Soup with Chicken and Snow Peas,1502.733236,12.179106,Low-Fat,"Coarse salt and ground pepper, 8 ounces rice n...",https://edamam-product-images.s3.amazonaws.com...,https://www.marthastewart.com/314010/asian-noo...
3552,"food_bknlkyzbuzo27pb11whr4bttkuy6, food_a50yq9...",http://www.edamam.com/ontologies/edamam.owl#re...,Oven-Barbecued Asian Chicken,2773.401250,20.436880,Low-Carb,"1 bunch scallions, ¼ cup hoisin sauce, 1 table...",https://edamam-product-images.s3.amazonaws.com...,https://www.eatingwell.com/recipe/252400/oven-...


In [3]:
# TF-IDF based content filtering
tfidf = TfidfVectorizer(stop_words="english")
menu_df['ingredients'] = menu_df['ingredients'].fillna("")
menu_df['combined_features'] = menu_df['ingredients'] + " " + menu_df['diet_labels']
tfidf_matrix = tfidf.fit_transform(menu_df['combined_features'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(menu_df.index, index=menu_df['recipe_name']).drop_duplicates()

In [4]:
def get_recommendations_tfidf(input_text, cosine_sim=cosine_sim):
    input_tfidf = tfidf.transform([input_text])
    sim_scores = linear_kernel(input_tfidf, tfidf_matrix).flatten()
    return sim_scores

In [5]:
user_df = pd.read_csv("/kaggle/input/user-in-df/user.csv")
user_df = pd.concat([user_df]*int(np.ceil(len(menu_df)/len(user_df))), ignore_index=True)
user_df = user_df.iloc[:len(menu_df)]
user_df.head()

,user_id,age,height (cm),weight (kg),diabetes_history,diabetes_heritage,preferred_food,diet_labels,sugar_intake (g/day),calorie_intake (kcal/day)
0,1,25,175,70,No,No,Pizza,Balanced,50,2500
1,2,34,160,65,Yes,Yes,Salad,Vegan,25,2000
2,3,29,180,80,No,No,Sushi,Low-Carb,50,2200
3,4,42,170,75,Yes,Yes,Pasta,High-Protein,25,2300
4,5,55,165,85,Yes,No,Burger,Low-Fat,25,1800


In [6]:
menu_cat_features = menu_df[['diet_labels', 'recipe_name']]
user_cat_features = user_df[['diet_labels', 'preferred_food']]
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
menu_cat_encoded = ohe.fit_transform(menu_cat_features)
user_cat_encoded = ohe.fit_transform(user_cat_features)
menu_features_encoded = np.hstack([menu_cat_encoded]).astype(np.float32)
user_features_encoded = np.hstack([user_cat_encoded]).astype(np.float32)


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
X_menu_train, X_menu_test, X_user_train, X_user_test = train_test_split(
    menu_features_encoded, user_features_encoded, test_size=0.2, random_state=32)
y = np.random.randint(0, 2, len(X_menu_train) + len(X_menu_test)).astype(np.float32)
y_train = y[:len(X_menu_train)]
y_test = y[len(X_menu_train):]

input_menu = Input(shape=(menu_features_encoded.shape[1],), name='menu_input')
input_user = Input(shape=(user_features_encoded.shape[1],), name='user_input')

# Menu branch
menu_layer = Dense(128, activation='relu')(input_menu)
menu_layer = Dropout(0.5)(menu_layer)
menu_layer = BatchNormalization()(menu_layer)
menu_layer = Dense(256, activation='relu')(input_menu)
menu_layer = Dropout(0.5)(menu_layer)
menu_layer = BatchNormalization()(menu_layer)

# User branch
user_layer = Dense(128, activation='relu')(input_user)
user_layer = Dropout(0.5)(user_layer)
user_layer = BatchNormalization()(user_layer)
user_layer = Dense(256, activation='relu')(input_user)
user_layer = Dropout(0.5)(user_layer)
user_layer = BatchNormalization()(user_layer)

# Concatenate the outputs of the two branches
concatenated = Concatenate()([menu_layer, user_layer])

# Add more layers
x = Dense(256, activation='relu')(concatenated)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
output = Dense(1, activation='sigmoid')(x)  # Assuming binary classification
rms = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

# Build and compile the model
model = Model(inputs=[input_menu, input_user], outputs=output)
model.compile(optimizer=rms, loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
history = model.fit([X_menu_train, X_user_train], y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.5071 - loss: 0.8751 - val_accuracy: 0.4903 - val_loss: 0.7065
Epoch 2/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5122 - loss: 0.8552 - val_accuracy: 0.4903 - val_loss: 0.7224
Epoch 3/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4935 - loss: 0.8671 - val_accuracy: 0.4903 - val_loss: 0.7290
Epoch 4/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.4855 - loss: 0.8572 - val_accuracy: 0.4903 - val_loss: 0.7259
Epoch 5/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5097 - loss: 0.8247 - val_accuracy: 0.4868 - val_loss: 0.7160
Epoch 6/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5256 - loss: 0.8236 - val_accuracy: 0.4868 - val_loss: 0.7126
Epoch 7/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.5298 - loss: 0.8132 - val_accuracy: 0.4833 - val_loss: 0.7140
Epoch 8/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5180 - loss: 0.8198 - val_accuracy: 0.

In [8]:
# Function to get deep learning recommendations
def get_recommendations_deep_learning(user_input):
    new_user_cat_features = pd.DataFrame([{
        'diet_labels': user_input['diet_labels'],
        'preferred_food': user_input['preferred_food']
    }])
    new_user_cat_encoded = ohe.transform(new_user_cat_features)
    new_user_features_encoded = np.hstack([new_user_cat_encoded]).astype(np.float32)
    new_user_features_encoded = np.tile(new_user_features_encoded, (len(menu_features_encoded), 1))
    predictions = model.predict([menu_features_encoded, new_user_features_encoded])
    return predictions.flatten()

In [9]:
# Integrate both models
def get_combined_recommendations(input_text, user_input, weight_tfidf=0.5, weight_deep_learning=0.5):
    tfidf_scores = get_recommendations_tfidf(input_text)
    deep_learning_scores = get_recommendations_deep_learning(user_input)
    
    # Normalize scores
    tfidf_scores = (tfidf_scores - tfidf_scores.min()) / (tfidf_scores.max() - tfidf_scores.min())
    deep_learning_scores = (deep_learning_scores - deep_learning_scores.min()) / (deep_learning_scores.max() - deep_learning_scores.min())
    
    # Combine scores
    combined_scores = weight_tfidf * tfidf_scores + weight_deep_learning * deep_learning_scores
    
    menu_df['combined_score'] = combined_scores
    top_n = 10
    top_recommendations = menu_df.sort_values(by='combined_score', ascending=False).head(top_n)
    
    return top_recommendations[['recipe_name', 'diet_labels', 'combined_score']]

In [10]:
new_user_input = {
    'diet_labels': 'low-carb',
    'preferred_food': 'chicken'
}
title = {'recipe_name': 'Pineapple Angel Food Cake'}

In [11]:
# Example usage
input_text = "chicken spicy low-carb"
user_input = {
    'diet_labels': 'Low-Sugar',
    'preferred_food': 'ayam geprek'
}
recommendations = get_combined_recommendations(input_text, user_input)
print("Top Recommendations:")
print(recommendations)

112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Top Recommendations:
                                 recipe_name           diet_labels  \
567                    Jamaican jerk chicken              Low-Carb   
2382            Spicy & Healthy Chicken Soup            High-Fiber   
574       Grilled Butterflied Chicken Recipe              Low-Carb   
2379    Salmon Steaks with Spicy Paprika Rub  Low-Carb, Low-Sodium   
624                  Chicken alla Puttanesca              Low-Carb   
2423                  Spicy BBQ Wings Recipe              Low-Carb   
597   Cook the Book: Perfect Roasted Chicken              Low-Carb   
2388  Spicy Shallot Roasted Brussels Sprouts  High-Fiber, Low-Carb   
2459       Meat balls cooked in tomato sauce  Low-Carb, Low-Sodium   
2465         Sweet and Spicy Braised Chicken              Low-Carb   

      combined_score  
567         0.998124  
2382        0.932927  
574         0.909538  
2379        0.868076  
624         0.862657  
2423        0.860062  
597   

In [14]:
model.save('CBF_Recommend_System_model.h5')